In [2]:
import pandas as pd 

In [3]:
from sqlparsemodel.Sqlstructure import Sqlstructure
from sqlparsemodel.Sastructure import Sastructure
from sqlparsemodel.Mermaidplot import Sqltoflowchart
from sqlparsemodel.Querystate import Querystate
import re 
Querystate = Querystate()

In [6]:
query = '''


PROC SQL;
   CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
QUIT;



PROC SQL;
   CREATE TABLE WORK.PAST_CLAIM_ICDS AS 
   SELECT DISTINCT 
          t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO, 
          t2.DIAG_SER_NO, 
          t2.CFM_ICD_CODE1, 
          t2.CFM_ICD_CODE2, 
          t2.CFM_ICD_CODE3, 
          t2.CFM_ICD_CODE4, 
          t2.CFM_ICD_CODE5
      FROM MEOW.PAST_CLAIM_APLY_NO t1
	       INNER JOIN (SELECT * FROM QUERY.DTAAA020_AE) t2 ON (t1.APLY_NO = t2.APLY_NO);
QUIT;'''

query = '''


PROC SQL;
   CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
QUIT;



PROC SQL;
   CREATE TABLE WORK.PAST_CLAIM_ICDS AS 
   SELECT DISTINCT 
          t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO, 
          t2.DIAG_SER_NO, 
          t2.CFM_ICD_CODE1, 
          t2.CFM_ICD_CODE2, 
          t2.CFM_ICD_CODE3, 
          t2.CFM_ICD_CODE4, 
          t2.CFM_ICD_CODE5
      FROM MEOW.PAST_CLAIM_APLY_NO t1
	       INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO);
QUIT;'''

query3 = '''
PROC SQL;
   CREATE TABLE Meow.product_mkt_type_cd AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
          t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1'
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;
PROC SQL;
   CREATE TABLE Meow.Main_policy AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
                 t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1' AND t1.BASE_COVERAGE_INDICATOR_CD='1'
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;
PROC SQL;
   CREATE TABLE Meow.ADD_policy AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
          t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1' AND t1.BASE_COVERAGE_INDICATOR_CD='2'
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;
PROC SQL;
   CREATE TABLE Meow.ALL_policy AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
          t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1' 
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;

'''

query4 = '''

PROC SQL;
   CREATE TABLE Meow.HOSPITAL_5Y_01 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
         t1.CREATE_DATE,
          t1.INSD_ID_SAS,
          t2.APLY_NO,
		  t2.POLICY_NO AS POL_2, 
		  t2.PROD_ID,
	      t2.OCR_DATE,
		  t2.E_STR_DATE,
		  t2.E_END_DATE,
		  t2.CLAM_AMT_CODE,
		  t2.CLAM_AMT_NAME,
		  t2.CLAM_CAT,
		  t2.OCR_RESN, 
          t2.BONE_CODE, 
          t2.BONE_NAME, 
		  t2.OP_CODE,
          t2.OP_NAME,
		  t2.PAY_DAY,
		  t2.PAY_AMT,
		  t2.PAY_STS
      FROM Meow.POL_ALL_X_01 t1
           LEFT JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.ISSUE_DATE ; /*因為後面會使用OCR_DATE、STR_DATE跟END_DATE(先篩OCR_DATE會有點問題)，先篩投保時間稍微減少資料量*/
QUIT;
/*先用Meow.HOSPITAL_5Y_01的檔案，因為每次從理賠檔裡撈資料太久*/
PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_02 AS 
   SELECT DISTINCT  t1.CONTRACT_NO, 
           t1.CREATE_DATE, 
          t1.INSD_ID_SAS,
          t1.APLY_NO,
		  t1.POL_2 , 
		  t1.PROD_ID,
	      t1.OCR_DATE,
		  t1.E_STR_DATE,
		  t1.E_END_DATE,
		  t1.OCR_RESN, 
          t1.BONE_CODE, 
          t1.BONE_NAME, 
		   t1.CLAM_AMT_CODE,
		  t1.CLAM_AMT_NAME,
		  t1.OP_CODE,
          t1.OP_NAME,
		  t1.PAY_DAY
      FROM Meow.HOSPITAL_5Y_01 t1
      WHERE t1.PAY_AMT > 0 AND t1.PAY_STS NOT = '5'  AND CLAM_AMT_CODE in( 'BEE1' , 'BEF1') AND  t1.CLAM_CAT in ('E' ,'F') AND t1.BONE_CODE ='' ;
QUIT;

%_eg_conditional_dropds(WORK.QUERY_FOR_HOSPITAL_5Y_03_1);
PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_03 AS 
   SELECT DISTINCT  t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
           t1.CREATE_DATE, 
          t1.POL_2, 
          t1.OCR_DATE, 
		  t1.E_STR_DATE,
		  t1.E_END_DATE,
		    /* E_STR_DATE_NEW */
		  (CASE
         WHEN t1.E_STR_DATE NOT IS MISSING THEN t1.E_STR_DATE
          ELSE t1.OCR_DATE END)  FORMAT=date9. AS E_STR_DATE_NEW,
  /* E_END_DATE_NEW */
		(CASE 
		WHEN t1.E_STR_DATE NOT IS MISSING THEN t1.E_STR_DATE
		ELSE t1.OCR_DATE
		END)+t1.PAY_DAY-1  FORMAT=date9. AS E_END_DATE_NEW,
		   t1.CLAM_AMT_CODE, 
          t1.CLAM_AMT_NAME,
		  t1.OCR_RESN, 
          t1.BONE_CODE, 
          t1.BONE_NAME,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_02 t1;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_0to1 AS 
   SELECT DISTINCT  t1.CONTRACT_NO,
         t1.INSD_ID_SAS, 
		  t1.CREATE_DATE,
          t1.POL_2, 
		  t1.OCR_RESN, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=   t1.CREATE_DATE-365*1 AND t1.E_STR_DATE_NEW <  t1.CREATE_DATE ;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_1to2 AS 
   SELECT DISTINCT t1.CONTRACT_NO,
         t1.INSD_ID_SAS, 
		  t1.CREATE_DATE,
          t1.POL_2, 
		  t1.OCR_RESN, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=  t1.CREATE_DATE-365*2 AND t1.E_STR_DATE_NEW <  t1.CREATE_DATE-365*1 ;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_2to3 AS 
   SELECT DISTINCT t1.CONTRACT_NO,
           t1.INSD_ID_SAS, 
		    t1.CREATE_DATE,
          t1.POL_2, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=   t1.CREATE_DATE-365*3 AND t1.E_STR_DATE_NEW <   t1.CREATE_DATE-365*2 ;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_3to4 AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS, 
		   t1.CREATE_DATE,
          t1.POL_2, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=   t1.CREATE_DATE-365*4 AND t1.E_STR_DATE_NEW <  t1.CREATE_DATE-365*3 ;
QUIT;
PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_4to5 AS 
   SELECT DISTINCT t1.CONTRACT_NO,
         t1.INSD_ID_SAS, 
		  t1.CREATE_DATE,
          t1.POL_2, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=   t1.CREATE_DATE-365*5 AND t1.E_STR_DATE_NEW <   t1.CREATE_DATE-365*4 ;
QUIT;
 PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_05_year_0to1 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
         t1.INSD_ID_SAS, 
		 t1.POL_2, 
          t1.E_STR_DATE_NEW, 
          t1.E_END_DATE_NEW, 
          /* NEW_PAY_DAY */
            (MIN( t1.CREATE_DATE-1,t1.E_END_DATE_NEW)- MAX( t1.CREATE_DATE-365*1,t1.E_STR_DATE_NEW)+1) AS NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_04_year_0to1 t1;
QUIT;
 PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_05_year_1to2 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
         t1.INSD_ID_SAS, 
		 t1.POL_2, 
          t1.E_STR_DATE_NEW, 
          t1.E_END_DATE_NEW, 
          /* NEW_PAY_DAY */
            (MIN( t1.CREATE_DATE-365*1-1,t1.E_END_DATE_NEW)- MAX( t1.CREATE_DATE-365*2,t1.E_STR_DATE_NEW)+1) AS NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_04_year_1to2 t1;
QUIT;
 PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_05_year_2to3 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
         t1.INSD_ID_SAS, 
		 t1.POL_2, 
          t1.E_STR_DATE_NEW, 
          t1.E_END_DATE_NEW, 
          /* NEW_PAY_DAY */
            (MIN( t1.CREATE_DATE-365*2-1,t1.E_END_DATE_NEW)- MAX( t1.CREATE_DATE-365*3,t1.E_STR_DATE_NEW)+1) AS NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_04_year_2to3 t1;
QUIT;
 PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_05_year_3to4 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
         t1.INSD_ID_SAS, 
		 t1.POL_2, 
          t1.E_STR_DATE_NEW, 
          t1.E_END_DATE_NEW, 
          /* NEW_PAY_DAY */
            (MIN( t1.CREATE_DATE-365*3-1,t1.E_END_DATE_NEW)- MAX( t1.CREATE_DATE-365*4,t1.E_STR_DATE_NEW)+1) AS NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_04_year_3to4 t1;
QUIT;
 PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_05_year_4to5 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
         t1.INSD_ID_SAS, 
		 t1.POL_2, 
          t1.E_STR_DATE_NEW, 
          t1.E_END_DATE_NEW, 
          /* NEW_PAY_DAY */
            (MIN( t1.CREATE_DATE-365*4-1,t1.E_END_DATE_NEW)- MAX( t1.CREATE_DATE-365*5,t1.E_STR_DATE_NEW)+1) AS NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_04_year_4to5 t1;
QUIT;
%_eg_conditional_dropds(WORK.QUERY_FOR_HOSPITAL_5Y_03_1_q);

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_06_0to1 AS 
   SELECT DISTINCT t1.POL_2, 
          t1.CONTRACT_NO, 
          /* SUM_of_NEW_PAY_DAY */
            (SUM(t1.NEW_PAY_DAY)) AS SUM_of_NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_05_YEAR_0to1 t1
	  WHERE t1.NEW_PAY_DAY>0
      GROUP BY t1.POL_2,
               t1.CONTRACT_NO
      ORDER BY t1.CONTRACT_NO;

QUIT;
%_eg_conditional_dropds(WORK.QUERY_FOR_HOSPITAL_5Y_05_YE_0000);

PROC SQL;
   CREATE TABLE Meow.HOSPITAL_5Y_07_0to1 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
          /* MAX_of_SUM_of_NEW_PAY_DAY */
             (MAX(t1.SUM_of_NEW_PAY_DAY)) AS HSP_DAY_Y2015
      FROM WORK.HOSPITAL_5Y_06_0to1 t1
      GROUP BY t1.CONTRACT_NO;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_06_1to2 AS 
   SELECT DISTINCT t1.POL_2, 
          t1.CONTRACT_NO, 
          /* SUM_of_NEW_PAY_DAY */
            (SUM(t1.NEW_PAY_DAY)) AS SUM_of_NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_05_YEAR_1to2 t1
WHERE t1.NEW_PAY_DAY>0
      GROUP BY t1.POL_2,
               t1.CONTRACT_NO
      ORDER BY t1.CONTRACT_NO;
QUIT;
%_eg_conditional_dropds(WORK.QUERY_FOR_HOSPITAL_5Y_05_YE_0000);

PROC SQL;
   CREATE TABLE Meow.HOSPITAL_5Y_07_1to2 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
          /* MAX_of_SUM_of_NEW_PAY_DAY */
            (MAX(t1.SUM_of_NEW_PAY_DAY)) AS HSP_DAY_Y2014
      FROM WORK.HOSPITAL_5Y_06_1to2 t1
      GROUP BY t1.CONTRACT_NO;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_06_2to3 AS 
   SELECT DISTINCT t1.POL_2, 
          t1.CONTRACT_NO, 
          /* SUM_of_NEW_PAY_DAY */
            (SUM(t1.NEW_PAY_DAY)) AS SUM_of_NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_05_YEAR_2to3 t1
WHERE t1.NEW_PAY_DAY>0
      GROUP BY t1.POL_2,
               t1.CONTRACT_NO
      ORDER BY t1.CONTRACT_NO;
QUIT;

PROC SQL;
   CREATE TABLE Meow.HOSPITAL_5Y_07_2to3 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
          /* MAX_of_SUM_of_NEW_PAY_DAY */
            (MAX(t1.SUM_of_NEW_PAY_DAY)) AS HSP_DAY_Y2013
      FROM WORK.HOSPITAL_5Y_06_2to3 t1
      GROUP BY t1.CONTRACT_NO;
QUIT;
PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_06_3to4 AS 
   SELECT DISTINCT t1.POL_2, 
          t1.CONTRACT_NO, 
          /* SUM_of_NEW_PAY_DAY */
            (SUM(t1.NEW_PAY_DAY)) AS SUM_of_NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_05_YEAR_3to4 t1
WHERE t1.NEW_PAY_DAY>0
      GROUP BY t1.POL_2,
               t1.CONTRACT_NO
      ORDER BY t1.CONTRACT_NO;
QUIT;

PROC SQL;
   CREATE TABLE Meow.HOSPITAL_5Y_07_3to4 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
          /* MAX_of_SUM_of_NEW_PAY_DAY */
            (MAX(t1.SUM_of_NEW_PAY_DAY)) AS HSP_DAY_Y2012
      FROM WORK.HOSPITAL_5Y_06_3to4 t1
      GROUP BY t1.CONTRACT_NO;
QUIT;
PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_06_4to5 AS 
   SELECT DISTINCT t1.POL_2, 
          t1.CONTRACT_NO, 
          /* SUM_of_NEW_PAY_DAY */
            (SUM(t1.NEW_PAY_DAY)) AS SUM_of_NEW_PAY_DAY
      FROM WORK.HOSPITAL_5Y_05_YEAR_4to5 t1
WHERE t1.NEW_PAY_DAY>0
      GROUP BY t1.POL_2,
               t1.CONTRACT_NO
      ORDER BY t1.CONTRACT_NO;
QUIT;

PROC SQL;
   CREATE TABLE Meow.HOSPITAL_5Y_07_4to5 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
          /* MAX_of_SUM_of_NEW_PAY_DAY */
            (MAX(t1.SUM_of_NEW_PAY_DAY)) AS HSP_DAY_Y2011
      FROM WORK.HOSPITAL_5Y_06_4to5 t1
      GROUP BY t1.CONTRACT_NO;
QUIT;



'''

query5 = '''

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_1to2 AS 
   SELECT DISTINCT t1.CONTRACT_NO,
         t1.INSD_ID_SAS, 
		  t1.CREATE_DATE,
          t1.POL_2, 
		  t1.OCR_RESN, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=  t1.CREATE_DATE-365*2 AND t1.E_STR_DATE_NEW <  t1.CREATE_DATE-365*1 ;
QUIT;


'''

query6 = '''
SELECT * From 
(
  SELECT  *, (Dense_Rank() OVER (ORDER BY Column ASC)) AS Rank, b
  FROM Table1 
) AS Ranking 
WHERE Rank % 10 = 0;

'''


query7 = '''

PROC SQL;
   CREATE TABLE Meow.SPCL_DTATA007_X_00 AS 
   SELECT t2.CONTRACT_NO,
         (CATX( "_" ,t1.REFUSE_FILENO , t1.CODE_CAT , t1.APPROVE_CODE)) AS unique_code, 
         t2.CREATE_DATE, 
          t2.REFUSE_FILENO,
          t2.APPLY_NO, 
		  t2.UPDATE_DATE, 
          t2.ID_SAS AS INSD_ID_SAS, 
          t2.POLICY_NO, 
          t2.KIND,
          t1.CODE_CAT, 
          t1.APPROVE_CODE, 
          t1.APPROVE_DETAIL, 
          t1.MEMO_PUBLIC, 
          t1.MEMO_PRIVATE
        FROM WORK.SPCL_DTATA007_X_00 merge WORK.SPCL_DTATA007_X_01
QUIT;    

data SPECIAL_to_icd_03;
merge SPECIAL_to_icd_03_1 SPECIAL_to_icd_03_2;
run  ;

data WORK.MED_EXAM_03;
set WORK.MED_EXAM_02;
by CONTRACT_NO;
if last.CONTRACT_NO then output; 
run  ;

PROC SORT DATA=QUERY.DTAAB001_Q OUT=WORK.DTAAB001_Q_S;
BY OCR_ID_SAS POLICY_NO PROD_ID;
RUN;

proc transpose data=WORK.SPECIAL_to_icd_02_2 out=WORK.SPECIAL_to_icd_03_2;
	var CONFIDENCE_LEVEL_1-CONFIDENCE_LEVEL_5;
  	by unique_code;
run;

proc transpose data=WORK.SPECIAL_to_icd_02_1 out=WORK.SPECIAL_to_icd_03_1(rename=(col1=ICD_CODE));
	var ICD_CODE_1-ICD_CODE_5;
  	by unique_code;
run;

'''

query8 = '''

PROC SQL;
CREATE TABLE A AS 
SELECT * FROM POL_USE WHERE EFT_CODE='46'
;
QUIT;
PROC SQL;
CREATE TABLE TEMP_FINAL_PAYDAY AS 
SELECT b.* 
FROM QUERY.DTAAB001_Q as b
INNER JOIN  a
		on a.PROD_ID=b.PROD_ID AND a.POLICY_NO=b.POLICY_NO AND a.PROD_ID=b.PROD_ID
WHERE PAY_STS not in ('0','5','6') AND CLAM_CAT in('B','C','E','F','G','H','N')
		
;
QUIT;

PROC SQL;
CREATE TABLE FINAL_PAYDAY AS 
SELECT	POLICY_NO,
		PROD_ID,
		OCR_ID_SAS,
		max(ACNT_DATE) FORMAT=YYMMDD10. AS 最後給付日 

FROM TEMP_FINAL_PAYDAY
group by 1,2,3
;
DROP TABLE A;
DROP TABLE TEMP_FINAL_PAYDAY;
QUIT;

/*：抓死殘的日期*/
/*理賠記錄檔*/
						 /*契約角色(0：子女 C：配偶 I：被保人 A：要保人 S：次被保人)*/
						 /*a.CLAM_CAT為索賠類別   a.PAY_STS為給付狀態   */
						/*PAY_STS 欄位名稱說明0 尚未給付 1 依約給付 2 協議給付 3 削減給付	4 修正延滯息日期 
						5 退件不給付 6 暫不處理 7 人工處理 
						CLAM_CAT 欄位名稱說明	A 死亡 B 殘廢 C 重大疾病 D 生命末期 E 醫療日額 F 醫療實支 G 防癌 
						H 長期看護 	I 婦女津貼 	J 豁免保費 	K 全殘 L 解除契約 N 定期給付保險金 	Z 延滯息 */
/**【身故全殘日整理】**/
/* 理賠保險金代號：
    BBA*：死亡保險金-->這種判斷法未來可能會被調整，有風險,就像險別當初兩碼轉三碼專案一樣，未來OB1跟OB2不見得是同一系列險種
    BBK*：全殘保險金
	醫療險全殘還在，壽險死殘日要再額外抓CLAM_CAT是K的來看
/*新用法，做成ID對照日期，判斷日期時不用篩選給付狀態056，因為真的身故但依約不理賠也是可信的日期
	資服說END_DATE是改變契約效力用的日期，抽驗過事故日期與END_DATE不同的資料也是END_DATE符合死亡證明日期*/
/*死亡全殘後續用ID比對就可以，理論上一個人死亡全殘的日期為固定日期是合理的*/

PROC SQL;/*先跑出要算哪些保單的身故全殘日*/
CREATE TABLE a AS 
SELECT * FROM POL_USE WHERE EFT_CODE in ('14','38','40','42','50','51','44','47')
;
QUIT;
PROC SQL;/*先篩選出該保單被保人ID的所有身故理賠資料*/
CREATE TABLE b AS 
SELECT DTAAB001_Q.* 
FROM QUERY.DTAAB001_Q
INNER JOIN a AS POL_USE ON DTAAB001_Q.OCR_ID_SAS=POL_USE.INSD_ID_SAS
				WHERE SYS_NO='1' AND CLAM_CAT IN ('A')
;
QUIT;
PROC SQL;/*先篩選出該保單被保人ID的所有全殘理賠資料*/
CREATE TABLE c AS 
SELECT DTAAB001_Q.* 
FROM QUERY.DTAAB001_Q
INNER JOIN a AS POL_USE ON DTAAB001_Q.OCR_ID_SAS=POL_USE.INSD_ID_SAS
				WHERE SYS_NO='1' AND CLAM_CAT IN ('K') AND PAY_STS not in ('0','5','6')
;
QUIT;
PROC SQL;/*作出每個ID對應身故全殘理賠日期*/
CREATE TABLE d AS 
SELECT DISTINCT
	 a.INSD_ID_SAS,
	 b.END_DATE FORMAT=YYMMDD10. AS 身故日,
     b.OCR_DATE FORMAT=YYMMDD10. AS 身故資料事故日,
	 c.END_DATE FORMAT=YYMMDD10. AS 全殘日,
	 c.OCR_DATE FORMAT=YYMMDD10. AS 全殘資料事故日
FROM a
LEFT JOIN b on a.INSD_ID_SAS=b.OCR_ID_SAS
LEFT JOIN c on a.INSD_ID_SAS=c.OCR_ID_SAS
;
QUIT;
/*作出每個ID對應身故全殘理賠日期*/
/*若ENDDAY都取到空白的才取事故日當作身故/全殘日*/
PROC SQL;
CREATE TABLE D01 AS 
SELECT INSD_ID_SAS,
		CASE WHEN 身故日=. THEN 身故資料事故日
		else 身故日
		END AS 身故日 FORMAT=YYMMDD10.,
		CASE WHEN 全殘日=. THEN 全殘資料事故日
		else 全殘日
		END AS 全殘日 FORMAT=YYMMDD10.
FROM(
	SELECT 
		 INSD_ID_SAS,
	 	 min(身故日) FORMAT=YYMMDD10. AS 身故日,
		 min(身故資料事故日) FORMAT=YYMMDD10. AS 身故資料事故日,
		 min(全殘日) FORMAT=YYMMDD10. AS 全殘日,
		 min(全殘資料事故日) FORMAT=YYMMDD10. AS 全殘資料事故日
	FROM d 
	GROUP BY 1)
;
QUIT;


'''

query9 = '''

proc sort data=WORK.QUERY_FOR_M_TYPE_ALL_0004
OUT=WORK.CLAM_CAT_T; 
by APLY_NO CLAM_CAT;RUN;
PROC TRANSPOSE DATA=WORK.CLAM_CAT_T 
OUT=WORK.CLAM_CAT_T2 (drop=_name_);
var VALUE;
by APLY_NO;
id CLAM_CAT;
run; 






'''
query = re.sub("\s+?;",";",query9)
query = re.sub("(\n)|(\r)|(\t)"," ",query)
query = re.sub("\\/\\*.*?\\*\\/", "",query)

# query to Structure List to SAS node relation 
sas_query = Sastructure(query)

# Node relation Setting and Plot the mermaid structure
sfc = Sqltoflowchart(sas_query.from_to_property,
 sas_query.from_to_query,
 sas_query.node_property )

# 1. Ignore Some Relation words 
# 2. Draw some node to colors 
# 3. Draw the query to colors 
sfc.mermaid_plot(relation_ignore = [])
## FOR SAS CODE 
sfc.mermaid_drawnode_containtxt(keytxt = "DATA_STEP_FLOWCHART", colname = "state_value", color = "#00FFFF", allquery_bool = True)
sfc.mermaid_drawnode_containtxt(keytxt = "PROC_TRANSPOSE_FLOWCHART", colname = "state_value", color = "#00FFFF", allquery_bool = True)
sfc.mermaid_drawnode_containtxt(keytxt = "PROC_SORT_FLOWCHART", colname = "state_value", color = "#00FFFF", allquery_bool = True)
## FOR ALL CODE
sfc.mermaid_drawnode(keyword = "WORK", token_tag = "token", color = "#F0BBFF", equalkeyword = False)
sfc.mermaid_drawnode(keyword = "QUERY", token_tag = "token", color = "#28FF28")
sfc.mermaid_drawnode(keyword = "CR20", token_tag = "token", color = "#FFAA33")
sfc.mc.mermaid_txt = re.sub("\n|\r", "",sfc.mc.mermaid_txt)        
sfc.mc.mermaid_txt = re.sub(";", ";\n",sfc.mc.mermaid_txt)
print(sfc.mc.mermaid_txt)

3. proc_transpose=================
{}
|- 0 DDL 'CREATE'
|- 1 Whitespace ' '
|- 2 Keyword 'TABLE'
|- 3 Whitespace ' '
|- 4 Identifier 'WORK.C...'
|  |- 0 Name 'WORK'
|  |- 1 Punctuation '.'
|  `- 2 Name 'CLAM_C...'
|- 5 Whitespace ' '
|- 6 Whitespace ' '
|- 7 DDL 'drop'
|- 8 Comparison '='
|- 9 Identifier '_name_'
|  `- 0 Name '_name_'
|- 10 Whitespace ' '
|- 11 Keyword 'FROM'
|- 12 Whitespace ' '
|- 13 Identifier 'WORK.C...'
|  |- 0 Name 'WORK'
|  |- 1 Punctuation '.'
|  |- 2 Name 'CLAM_C...'
|  |- 3 Whitespace ' '
|  `- 4 Identifier 'AND'
|     `- 0 Name 'AND'
`- 14 Parenthesis '(PROC_...'
   |- 0 Punctuation '('
   |- 1 Identifier 'PROC_T...'
   |  `- 0 Name 'PROC_T...'
   `- 2 Punctuation ')'
None
SELECT表示句:TABLE
FROM表示句:FROM
1
SELECT表示句:TABLE
FROM表示句:FROM
4. proc_sort=================
{}
|- 0 DDL 'CREATE'
|- 1 Whitespace ' '
|- 2 Keyword 'TABLE'
|- 3 Whitespace ' '
|- 4 Identifier 'WORK.C...'
|  |- 0 Name 'WORK'
|  |- 1 Punctuation '.'
|  `- 2 Name 'CLAM_C...'
|- 5 Whitespace ' '
|

In [6]:
sas_query.node_dict

{'1': ['1',
  '1',
  'MAIN',
  'FROM',
  'value',
  'WORK.QUERY_FOR_M_TYPE_ALL_0004 AND (PROC_SORT_FLOWCHART)',
  '',
  '',
  ''],
 '2': ['2',
  '1',
  'MAIN',
  'FROM',
  'token',
  'alias:AND, Database:WORK, Table:QUERY_FOR_M_TYPE_ALL_0004',
  'AND',
  'WORK',
  'QUERY_FOR_M_TYPE_ALL_0004'],
 '3': ['3',
  '1',
  'MAIN',
  'TABLE',
  'token',
  'WORK.CLAM_CAT_T',
  '',
  'WORK',
  'CLAM_CAT_T']}

In [7]:
sas_query.node_property

[['node_id',
  'query_id',
  'subq_name',
  'state',
  'token_tag',
  'state_value',
  'state_alias',
  'state_parentname',
  'state_realname'],
 ['1',
  '1',
  'MAIN',
  'FROM',
  'value',
  'WORK.QUERY_FOR_M_TYPE_ALL_0004 AND (PROC_SORT_FLOWCHART)',
  '',
  '',
  ''],
 ['2',
  '1',
  'MAIN',
  'FROM',
  'token',
  'alias:AND, Database:WORK, Table:QUERY_FOR_M_TYPE_ALL_0004',
  'AND',
  'WORK',
  'QUERY_FOR_M_TYPE_ALL_0004'],
 ['3',
  '1',
  'MAIN',
  'TABLE',
  'token',
  'WORK.CLAM_CAT_T',
  '',
  'WORK',
  'CLAM_CAT_T']]

In [17]:
sas_query.sql_parsed_list["1"].replaced_dict

{'MAIN': '   CREATE TABLE DD01 AS SELECT POLICY_NO, \t   PROD_ID, \t   INSD_ID_SAS, \t   理賠金代號,        CASE WHEN 初次殘疾鑑定日<>. THEN 初次殘疾鑑定日 \t\t\tWHEN 初次殘疾鑑定日=. THEN 事故日 \t   END AS 初次殘疾鑑定日 FORMAT=YYMMDD10. FROM SUBQ.MAIN_FROM1 WHERE 初次殘疾鑑定日<>. OR 事故日<>.;   ',
 'MAIN_FROM1': ' \tSELECT \tDISTINCT  \t\t\tPOLICY_NO, \t\t\tPROD_ID, \t\t\tINSD_ID_SAS, \t\t\t理賠金代號, \t\t\tmin(初次殘疾鑑定日) FORMAT=YYMMDD10. AS 初次殘疾鑑定日, \t\t\tmin(事故日) FORMAT=YYMMDD10. AS 事故日 \t\t\tFROMSUBQ.MAIN_FROM1_FROM1 \tGROUP BY 1,2,3,4 \t',
 'MAIN_FROM1_FROM1': " \t\t\t\tSELECT c.POLICY_NO, \t\t\t\t\t   c.PROD_ID, \t\t\t\t\t   b.理賠金代號, \t\t\t\t\t   a.INSD_ID_SAS, \t\t\t\t\t   c.DISB_DATE  FORMAT=YYMMDD10. AS 初次殘疾鑑定日, \t\t\t\t\t   c.OCR_DATE  FORMAT=YYMMDD10. AS 事故日 \t\t\t\tFROM QUERY.DTAAB001_Q as c \t\t\t\tINNER JOIN  b ON b.理賠金代號=c.CLAM_AMT_CODE AND c.PROD_ID=b.PROD_ID \t\t\t\tINNER JOIN\ta  ON c.OCR_ID_SAS=a.INSD_ID_SAS AND c.POLICY_NO=a.POLICY_NO \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tAND c.PROD_ID=a.PROD_ID \t\t\t\tWHERE c.PAY_S

In [5]:
re.split("\s+", "WORK.PAST_RESTRICTED_X04_AA_APC")

['WORK.PAST_RESTRICTED_X04_AA_APC']

In [4]:
def upper_lower_regrex(string):
        regex_txt = ""
        for c in string:
            if c == " " : 
                regex_txt += " "
            else:
                regex_txt += "[" + c.lower() + c.upper() + "]"
        return(regex_txt)

query8 = '''


PROC SQL;
   CREATE TABLE Meow.SPCL_DTATA007_X_00 AS 
   SELECT t2.CONTRACT_NO,
         (CATX( "_" ,t1.REFUSE_FILENO , t1.CODE_CAT , t1.APPROVE_CODE)) AS unique_code, 
         t2.CREATE_DATE, 
          t2.REFUSE_FILENO,
          t2.APPLY_NO, 
		  t2.UPDATE_DATE, 
          t2.ID_SAS AS INSD_ID_SAS, 
          t2.POLICY_NO, 
          t2.KIND,
          t1.CODE_CAT, 
          t1.APPROVE_CODE, 
          t1.APPROVE_DETAIL, 
          t1.MEMO_PUBLIC, 
          t1.MEMO_PRIVATE
        FROM QUERY.DTATA007_APPROVE_HISTORY t1
       RIGHT JOIN Meow.SPCL_DTATA006 t2 ON (t1.REFUSE_FILENO = t2.REFUSE_FILENO);
QUIT;


PROC SQL;
   CREATE TABLE WORK.SPECIAL_to_icd_02_1 AS 
   SELECT t1.unique_code, 
          t1.ICD_CODE_1, 
          t1.ICD_CODE_2, 
          t1.ICD_CODE_3, 
          t1.ICD_CODE_4, 
          t1.ICD_CODE_5
      FROM Meow.SPCL_DTATA009_ORI t1;
QUIT;
%_eg_conditional_dropds(WORK.SPECIAL_to_icd_02_2);

PROC SQL;
   CREATE TABLE WORK.SPECIAL_to_icd_02_2 AS 
   SELECT t1.unique_code, 
          t1.CONFIDENCE_LEVEL_1, 
          t1.CONFIDENCE_LEVEL_2, 
          t1.CONFIDENCE_LEVEL_3, 
          t1.CONFIDENCE_LEVEL_4, 
          t1.CONFIDENCE_LEVEL_5
      FROM Meow.SPCL_DTATA009_ORI t1;
QUIT   ;
proc transpose data=WORK.SPECIAL_to_icd_02_1 out=WORK.SPECIAL_to_icd_03_1(rename=(col1=ICD_CODE));
	var ICD_CODE_1-ICD_CODE_5;
  	by unique_code;
run;

proc transpose data=WORK.SPECIAL_to_icd_02_2 out=WORK.SPECIAL_to_icd_03_2;
	var CONFIDENCE_LEVEL_1-CONFIDENCE_LEVEL_5;
  	by unique_code;
run;

PROC SORT DATA=MEOW.HOUSEHOLD_05;
BY INSD_ID_SAS POLICY_NO1;
RUN;


data SPECIAL_to_icd_03;
merge SPECIAL_to_icd_03_1 SPECIAL_to_icd_03_2;
run  ;

data WORK.MED_EXAM_03;
set WORK.MED_EXAM_02;
by CONTRACT_NO;
if last.CONTRACT_NO then output; 
run  ;


/*串理賠資料*/
PROC SORT DATA=QUERY.DTAAB001_Q OUT=WORK.DTAAB001_Q_S;
BY OCR_ID_SAS POLICY_NO PROD_ID;
RUN;

data WORK.PAST_RESTRICTED_X05_AA_APC;
        set WORK.PAST_RESTRICTED_X04_AA_APC;
              PAST_RESTRICTED_CS_APC_num=input(PAST_RESTRICTED_CS_APC,2.);
run;


'''
count = 0
query8 = re.sub("\s+?;",";",query8)
query8 = re.sub("(\n)|(\r)"," ",query8)
proc_sql = re.findall(sfc.upper_lower_regrex("PROC") + "\s+" + sfc.upper_lower_regrex("SQL") + ".*?" + sfc.upper_lower_regrex("QUIT;"), query8)# Get PROC SQL be the SQL string
data_step = re.findall(sfc.upper_lower_regrex("DATA") + "\s+(?!=).*?" + sfc.upper_lower_regrex("RUN;"), query8) # If the query with "=" after "data" not the Data Step target
proc_transpose = re.findall(sfc.upper_lower_regrex("PROC") + "\s+" + sfc.upper_lower_regrex("TRANSPOSE") + ".*?" + sfc.upper_lower_regrex("RUN;"), query8)
proc_sort = re.findall(sfc.upper_lower_regrex("PROC") + "\s+" + sfc.upper_lower_regrex("SORT") + ".*?" + sfc.upper_lower_regrex("RUN;"), query8)
# PROC SQL
for r in proc_sql :
    print("1. proc_sql=================")
    print(r)

# Data Step 
for r in data_step :
    table_name = ""
    from_name = ""
    print("2. data_step=================")
    print(r)
    ## Get Table name and Out name
    ### if out name is empy -> same as table name
    table_name = re.findall(sfc.upper_lower_regrex("DATA") + "(.*?);", r)
    table_name = table_name[0].strip()
    from_name = re.findall(sfc.upper_lower_regrex("SET") + "(.*?);", r)
    if len(from_name) == 0 : # SET no using MERGE
        from_name = re.findall(sfc.upper_lower_regrex("MERGE") + "(.*?);", r)
    
        
    if len(from_name) == 0 :
        from_name = table_name
    else :
        from_name = from_name[0].strip()
    print(from_name)
    from_list = re.split("\s+", from_name)
    print(from_list)
#     if len(re.findall("\\.", table_name)) == 0 :
#         table_name = "WORK." + table_name
#     if len(re.findall("\\.", from_name)) == 0 :
#         from_name = "WORK." + from_name
    
    print(dbfromname_tosql(re.sub(";", "",table_name), re.sub(";", "",from_name)))
#     query_id += 1
#     table_list = re.split("\\.", table_name)
#     from_list = re.split("\\.", from_name)
    
    
# PROC TRANSPOSE    
for r in proc_transpose :
    table_name = ""
    from_name = ""
    print("3. proc_transpose=================")
    # data 
    ## Replace some special sign, ex: "(", ")" -> space
    ### TABLE NAME GET
    ### FROM NAME GET
    rtmp = re.sub("\\(|\\)", " ", r)
    # trans_sign = PROC\s+TRANSPOSE\s+DATA
    trans_sign = sfc.upper_lower_regrex("PROC") + "\s+" + sfc.upper_lower_regrex("TRANSPOSE")+ "\s+" + sfc.upper_lower_regrex("DATA")
    table_name = re.findall(trans_sign + "\s*?=(.*?)\s+", rtmp)
    table_name = table_name[0].strip()
    from_name = re.findall(trans_sign + ".*?" + sfc.upper_lower_regrex("out") + "\s*?=(.*?);", rtmp)
    if len(from_name) == 0 :
        from_name = table_name
    else :
        from_name = re.split(" ",re.sub(";", " ",from_name[0]).strip())[0]
    
#     if len(re.findall("\\.", table_name)) == 0 :
#         table_name = "WORK." + table_name
#     if len(re.findall("\\.", from_name)) == 0 :
#         from_name = "WORK." + from_name
    
    
    print(dbfromname_tosql(re.sub(";", "",table_name), re.sub(";", "",from_name)))
    
# PROC SORT    
for r in proc_sort :
    table_name = ""
    from_name = ""
    print("4. proc_sort=================")
    # data 
    ## Replace some special sign, ex: "(", ")" -> space
    ### TABLE NAME GET
    ### FROM NAME GET
    rtmp = re.sub("\\(|\\)", " ", r)
    sort_sign = sfc.upper_lower_regrex("PROC") + "\s+" + sfc.upper_lower_regrex("SORT")+ "\s+" + sfc.upper_lower_regrex("DATA")
    table_name = re.findall(sort_sign + "\s*?=(.*?)\s+", rtmp)
    table_name = table_name[0].strip()
    from_name = re.findall(sort_sign + ".*?" + sfc.upper_lower_regrex("out") + "\s*?=(.*?);", rtmp)
    if len(from_name) == 0 :
        from_name = table_name
    else :
        from_name = re.split(" ",re.sub(";", " ",from_name[0]).strip())[0]
    
#     if len(re.findall("\\.", table_name)) == 0 :
#         table_name = "WORK." + table_name
#     if len(re.findall("\\.", from_name)) == 0 :
#         from_name = "WORK." + from_name
    
    print(dbfromname_tosql(re.sub(";", "",table_name), re.sub(";", "",from_name)))

NameError: name 'sfc' is not defined

In [7]:
def dbfromname_tosql(dbname, from_name):
    tosql = "CREATE TABLE "+ dbname + " FROM "
    from_list = re.split("\s+", from_name)
    if len(from_list) > 1 :
        tosql += from_list[0] + " merge " + from_list[1] 
    else : 
        tosql += from_list[0]
    return( tosql )

In [14]:
re.split("\s+", "WORK.SPECIAL_to_icd_03_1 SPECIAL_to_icd_03_2")

['WORK.SPECIAL_to_icd_03_1', 'SPECIAL_to_icd_03_2']

In [32]:
xx = "proc transpose data=WORK.SPECIAL_to_icd_02_1 out=WORK.SPECIAL_to_icd_03_1(rename=(col1=ICD_CODE));"
re.findall(sfc.upper_lower_regrex("out") + "\s*?=(.*?);", r)[0]

'WORK.SPECIAL_to_icd_03_1(rename=(col1=ICD_CODE))'

In [19]:
re.sub("\\(", "", "(123456)")

'123456)'

In [24]:
'''t1.CONTRACT_NO,
t1.UPDATE_DATE'''

't1.CONTRACT_NO,\nt1.UPDATE_DATE'

In [86]:
re.findall("PROC(.*?\r|\r|\n)QUIT",'''PROC 123456789
123456789

                            QUIT''')

[]

In [73]:
print('''PROC 
123456789

                            QUIT''')

PROC 
123456789

                            QUIT


In [21]:
print(sfc.mc.mermaid_txt)

graph TD;subgraph Meow.SPCL_DTATA007_X_00;2(<center><b>FROM</b><br>QUERY.DTATA007_APPROVE_HISTORY, t1</center>)-->1{<center><b>FROM</b></center>};3(<center><b>FROM</b><br>Meow.SPCL_DTATA006, t2</center>)-->1{<center><b>FROM</b></center>};1{<center><b>FROM</b></center>}-->|<center>QUERY.DTATA007_APPROVE_HISTORY t1<b> <u>RIGHT JOIN</u> </b>Meow.SPCL_DTATA006 t2 ON <br> t1.REFUSE_FILENO = t2.REFUSE_FILENO  </center>|4{<center><b>SELECT</b></center>};4{<center><b>SELECT</b></center>}-->|"<center>t2.CONTRACT_NO, <br>(CATX( '_' , <br>t1.REFUSE_FILENO , <br> t1.CODE_CAT , <br> t1.APPROVE_CODE))<b> <u>AS</u> </b>unique_code, <br> t2.CREATE_DATE, <br> t2.REFUSE_FILENO, <br>t2.APPLY_NO, <br> t2.UPDATE_DATE, <br> t2.ID_SAS<b> <u>AS</u> </b>INSD_ID_SAS, <br> t2.POLICY_NO, <br> t2.KIND, <br>t1.CODE_CAT, <br> t1.APPROVE_CODE, <br> t1.APPROVE_DETAIL, <br> t1.MEMO_PUBLIC, <br> t1.MEMO_PRIVATE</center>"|5(<center><b>TABLE</b><br>Meow.SPCL_DTATA007_X_00, </center>);end;  style 2 fill:#28FF28;


In [9]:
"'_'"

"'_'"

In [ ]:
graph TD;subgraph Meow.SPCL_DTATA006;
2(<center><b>FROM</b><br>Meow.POL_ALL_X_01, t1</center>)-->1{<center><b>FROM</b></center>};
3(<center><b>FROM</b><br>QUERY.DTATA006_WEAKRECORD_Q, t2</center>)-->1{<center><b>FROM</b></center>};
1{<center><b>FROM</b></center>}-->|<center>Meow.POL_ALL_X_01 t1<b> <u>LEFT JOIN</u> </b>QUERY.DTATA006_WEAKRECORD_Q t2 ON <br> t1.INSD_ID_SAS = t2.ID_SAS </center>|4{<center><b>WHERE</b></center>};4{<center><b>WHERE</b></center>}-->|<center>WHERE t2.REFUSE_FILENO<b> <u>NOT</u> </b>= '' AND <br>t2.UPDATE_DATE < t1.CREATE_DATE</center>|5{<center><b>SELECT</b></center>};5{<center><b>SELECT</b></center>}-->|"<center>t1.CONTRACT_NO, <br> t1.CREATE_DATE, <br>t2.REFUSE_FILENO, <br>t2.APPLY_NO, <br> t2.UPDATE_DATE, <br> t2.ID_SAS, <br> t2.POLICY_NO, <br> t2.KIND</center>"|6(<center><b>TABLE</b><br>Meow.SPCL_DTATA006, </center>);
end; 
subgraph Meow.SPCL_DTATA007_X_00;
8(<center><b>FROM</b><br>QUERY.DTATA007_APPROVE_HISTORY, t1</center>)-->7{<center><b>FROM</b></center>};
9(<center><b>FROM</b><br>Meow.SPCL_DTATA006, t2</center>)-->7{<center><b>FROM</b></center>};
7{<center><b>FROM</b></center>}-->|<center>QUERY.DTATA007_APPROVE_HISTORY t1<b> <u>RIGHT JOIN</u> </b>Meow.SPCL_DTATA006 t2 ON <br> t1.REFUSE_FILENO = t2.REFUSE_FILENO  </center>|10{<center><b>SELECT</b></center>};
10{<center><b>SELECT</b></center>}-->|"<center>t2.CONTRACT_NO, <br>(CATX( '_' , <br>t1.REFUSE_FILENO , <br> t1.CODE_CAT , <br> t1.APPROVE_CODE))<b> <u>AS</u> </b>unique_code, <br> t2.CREATE_DATE, <br> t2.REFUSE_FILENO, <br>t2.APPLY_NO, <br> t2.UPDATE_DATE, <br> t2.ID_SAS<b> <u>AS</u> </b>INSD_ID_SAS, <br> t2.POLICY_NO, <br> t2.KIND, <br>t1.CODE_CAT, <br> t1.APPROVE_CODE, <br> t1.APPROVE_DETAIL, <br> t1.MEMO_PUBLIC, <br> t1.MEMO_PRIVATE</center>"|11(<center><b>TABLE</b><br>Meow.SPCL_DTATA007_X_00, </center>);
end;
6-.->9;
style 3 fill:#28FF28; 
style 8 fill:#28FF28;

In [4]:
x = pd.DataFrame(sas_query.node_property[1:], columns = sas_query.node_property[0])
pd.DataFrame(sas_query.node_property[1:], columns = sas_query.node_property[0])

,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
0,1,1,MAIN,FROM,value,"& BASE t1, QUERY.DTAAB001_Q t2",,,
1,2,1,MAIN,FROM,token,"alias:t1, Database:, Table:BASE",t1,,BASE
2,3,1,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAB001_Q",t2,QUERY,DTAAB001_Q
3,4,1,MAIN,WHERE,value,WHERE (t1.APLY_NO = t2.APLY_NO AND t1.APLY_DAT...,,,
4,5,1,MAIN,SELECT,value,"DISTINCT t1.OCR_ID, t1.OCR_DATE, ...",,,
5,6,1,MAIN,TABLE,token,WORK.CLAM_CAT,,WORK,CLAM_CAT


In [8]:
x["state"] = x["state"].str.lower()

In [9]:
x

,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
0,1,1,MAIN,from,value,"& BASE t1, QUERY.DTAAB001_Q t2",,,
1,2,1,MAIN,from,token,"alias:t1, Database:, Table:BASE",t1,,BASE
2,3,1,MAIN,from,token,"alias:t2, Database:QUERY, Table:DTAAB001_Q",t2,QUERY,DTAAB001_Q
3,4,1,MAIN,where,value,WHERE (t1.APLY_NO = t2.APLY_NO AND t1.APLY_DAT...,,,
4,5,1,MAIN,select,value,"DISTINCT t1.OCR_ID, t1.OCR_DATE, ...",,,
5,6,1,MAIN,table,token,WORK.CLAM_CAT,,WORK,CLAM_CAT


In [8]:
pd.DataFrame(sas_query.from_to_property)

,0,1
0,from,to
1,2,1
2,1,3
3,3,4


In [8]:
pd.DataFrame(sas_query.from_to_query)

,0,1
0,from,to
1,4,6


In [11]:
print(re.sub('\\"', "'",'456456"123"456456'))

456456'123'456456


In [19]:
''' ' ''' + ''' \"  '''

 '  "  


In [10]:
print("\\\\")

\\


In [15]:
sas_query.sql_parsed_list['1'].structured_dict

{'MAIN': {'': {'value': 'CREATE'},
  'TABLE': {'value': 'WORK.HOSPITAL_5Y_06_4to5',
   'token': [[None, 'WORK', 'HOSPITAL_5Y_06_4to5']]},
  'OTHER': {'value': 'AS'},
  'SELECT': {'value': 'DISTINCT t1.POL_2, t1.CONTRACT_NO, (SUM(t1.NEW_PAY_DAY)) AS SUM_of_NEW_PAY_DAY'},
  'FROM': {'value': 'WORK.HOSPITAL_5Y_05_YEAR_4to5 t1',
   'token': [['t1', 'WORK', 'HOSPITAL_5Y_05_YEAR_4to5']]},
  'WHERE': {'value': 'WHERE t1.NEW_PAY_DAY>0'},
  'GROUPBY': {'value': 't1.POL_2,t1.CONTRACT_NO'},
  'ORDERBY': {'value': 't1.CONTRACT_NO ;'}}}

In [37]:
sas_query.sql_parsed_list['1'].structured_dict['MAIN']['TABLE'] = {'token' : [['', 'QUERYID', 'query' + '1']], 'value': 'QUERYID.query' + '1'}

In [3]:
sas_query.sql_parsed_list['1'].structured_dict['MAIN']

{'FROM': {'token': [['Ranking', 'SUBQ', 'MAIN_FROM1']],
  'value': 'SUBQ.MAIN_FROM1 AS Ranking'},
 'SELECT': {'value': '*'},
 'TABLE': {'token': [['', 'QUERYID', 'query1']], 'value': 'QUERYID.query1'},
 'WHERE': {'value': 'WHERE Rank % 10 = 0;'}}

In [4]:
sas_query.sql_parsed_list['1'].structured_dict['MAIN_FROM1']

{'FROM': {'token': [[None, None, 'Table1']], 'value': 'Table1'},
 'SELECT': {'value': '*, (Dense_Rank() OVER (ORDER BY Column ASC)) AS Rank, b'},
 'TABLE': {'token': [['', 'SUBQ', 'MAIN_FROM1']], 'value': 'SUBQ.MAIN_FROM1'}}

In [20]:
from selenium import webdriver
driver = webdriver.Chrome() #開啟firefox

driver.get("https://mermaid-js.github.io/mermaid-live-editor/") #前往這個網址

# 尋找網頁中的搜尋框
inputElement = driver.find_element_by_xpath("//div[@class='view-lines']")
print(inputElement.text)
# 在搜尋框中輸入文字
inputElement.send_keys("Selenium")


# driver.close()#關閉瀏覽器

graph TD
  A[Christmas] -->|Get money| B(Go shopping)
  B --> C{Let me think}
  C -->|One| D[Laptop]
  C -->|Two| E[iPhone]
  C -->|Three| F[fa:fa-car Car]
    


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=83.0.4103.106)


In [21]:
re.sub("\\/.*?\\/", "", query4)

"\n\nPROC SQL;\n   CREATE TABLE Meow.HOSPITAL_5Y_01 AS \n   SELECT DISTINCT t1.CONTRACT_NO, \n         t1.CREATE_DATE,\n          t1.INSD_ID_SAS,\n          t2.APLY_NO,\n\t\t  t2.POLICY_NO AS POL_2, \n\t\t  t2.PROD_ID,\n\t      t2.OCR_DATE,\n\t\t  t2.E_STR_DATE,\n\t\t  t2.E_END_DATE,\n\t\t  t2.CLAM_AMT_CODE,\n\t\t  t2.CLAM_AMT_NAME,\n\t\t  t2.CLAM_CAT,\n\t\t  t2.OCR_RESN, \n          t2.BONE_CODE, \n          t2.BONE_NAME, \n\t\t  t2.OP_CODE,\n          t2.OP_NAME,\n\t\t  t2.PAY_DAY,\n\t\t  t2.PAY_AMT,\n\t\t  t2.PAY_STS\n      FROM Meow.POL_ALL_X_01 t1\n           LEFT JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)\n      WHERE t1.CREATE_DATE > t2.ISSUE_DATE ;\nQUIT;\n\nPROC SQL;\n   CREATE TABLE WORK.HOSPITAL_5Y_02 AS \n   SELECT DISTINCT  t1.CONTRACT_NO, \n           t1.CREATE_DATE, \n          t1.INSD_ID_SAS,\n          t1.APLY_NO,\n\t\t  t1.POL_2 , \n\t\t  t1.PROD_ID,\n\t      t1.OCR_DATE,\n\t\t  t1.E_STR_DATE,\n\t\t  t1.E_END_DATE,\n\t\t  t1.OCR_RESN, \n          t1.

In [16]:
query4

"\n\nPROC SQL;\n   CREATE TABLE Meow.HOSPITAL_5Y_01 AS \n   SELECT DISTINCT t1.CONTRACT_NO, \n         t1.CREATE_DATE,\n          t1.INSD_ID_SAS,\n          t2.APLY_NO,\n\t\t  t2.POLICY_NO AS POL_2, \n\t\t  t2.PROD_ID,\n\t      t2.OCR_DATE,\n\t\t  t2.E_STR_DATE,\n\t\t  t2.E_END_DATE,\n\t\t  t2.CLAM_AMT_CODE,\n\t\t  t2.CLAM_AMT_NAME,\n\t\t  t2.CLAM_CAT,\n\t\t  t2.OCR_RESN, \n          t2.BONE_CODE, \n          t2.BONE_NAME, \n\t\t  t2.OP_CODE,\n          t2.OP_NAME,\n\t\t  t2.PAY_DAY,\n\t\t  t2.PAY_AMT,\n\t\t  t2.PAY_STS\n      FROM Meow.POL_ALL_X_01 t1\n           LEFT JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)\n      WHERE t1.CREATE_DATE > t2.ISSUE_DATE ;\nQUIT;\n\nPROC SQL;\n   CREATE TABLE WORK.HOSPITAL_5Y_02 AS \n   SELECT DISTINCT  t1.CONTRACT_NO, \n           t1.CREATE_DATE, \n          t1.INSD_ID_SAS,\n          t1.APLY_NO,\n\t\t  t1.POL_2 , \n\t\t  t1.PROD_ID,\n\t      t1.OCR_DATE,\n\t\t  t1.E_STR_DATE,\n\t\t  t1.E_END_DATE,\n\t\t  t1.OCR_RESN, \n          t1.

In [3]:
sas_query.from_to_query

[['from', 'to'], ['6', '8']]

In [4]:
sas_query.from_to_property

[['from', 'to'],
 ['2', '1'],
 ['3', '1'],
 ['1', '4'],
 ['4', '5'],
 ['5', '6'],
 ['8', '7'],
 ['9', '7'],
 ['7', '10'],
 ['10', '11']]

In [5]:
for item in sas_query.node_dict.items():
    print(str(item[0]) + " : " + "Node_prop : " + str(item[1][3]) + "," + str(item[1][4]) + " \n    Node_value : " + str(item[1][5]))

1 : Node_prop : FROM,value 
    Node_value : Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
2 : Node_prop : FROM,token 
    Node_value : alias:, Database:Meow, Table:POL_ALL_X_01
3 : Node_prop : FROM,token 
    Node_value : alias:t2, Database:QUERY, Table:DTAAB001_Q
4 : Node_prop : WHERE,value 
    Node_value : WHERE t1.CREATE_DATE > t2.OCR_DATE ;
5 : Node_prop : SELECT,value 
    Node_value : DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO
6 : Node_prop : TABLE,token 
    Node_value : MEOW.PAST_CLAIM_APLY_NO
7 : Node_prop : FROM,value 
    Node_value : MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO) ;
8 : Node_prop : FROM,token 
    Node_value : alias:t1, Database:MEOW, Table:PAST_CLAIM_APLY_NO
9 : Node_prop : FROM,token 
    Node_value : alias:t2, Database:QUERY, Table:DTAAA020_AE
10 : Node_prop : SELECT,value 
    Node_value : DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO, t2.DIAG_SER_NO, t2.CFM_ICD_

In [2]:
import markdown
text = """
# Title

Some text.

~~~mermaid
graph TB
A --> B
B --> C
~~~

Some other text.

~~~mermaid
graph TB
D --> E
E --> F
~~~
"""

html = markdown.markdown(text, extensions=['md_mermaid'])

print(html)

<h1>Title</h1>
<p>Some text.</p>
<div class="mermaid">
graph TB
A --> B
B --> C
</div>

<p>Some other text.</p>
<div class="mermaid">
graph TB
D --> E
E --> F
</div>

<script>mermaid.initialize({startOnLoad:true});</script>


In [237]:
class Mermaid_creator():
    
    def __init__(self):
        self.mermaid_txt = "graph TD;"
    
    # 00 Close Sentence
    def close_sentence(self):
        self.mermaid_txt += ";"
    
    # 01 Add Node    
    def add_node(self, shape = "", txt = "", node_id = ""):
        # Setting node id 
        self.mermaid_txt += str(node_id)
        
        # Setting texts
        if shape == "rectangular":
            self.mermaid_txt += "[" + txt +"]"
            
        elif shape == "roundedges":
            self.mermaid_txt += "(" + txt +")"
            
        elif shape == "circle":
            self.mermaid_txt += "((" + txt +"))"
            
        elif shape == "tag":
            self.mermaid_txt += ">" + txt +"]"
            
        elif shape == "diamond":
            self.mermaid_txt += "{" + txt +"}" 
            
            
    # 02 Add Link        
    def add_link(self, shape = "", txt = ""):
        
        # Setting relation texts
        if shape == "":
            print("please set a shape for line")
        
        # 01 Arrowhead
        elif shape == "arrowhead" :
            if txt == "" : 
                self.mermaid_txt += "-->"
            else:
                self.mermaid_txt += "-->" + "|" + txt + "|"
        
        # 02 Openlink
        elif shape == "openlink" :
            if txt == "" : 
                self.mermaid_txt += "---"
            else:
                self.mermaid_txt += "---" + "|" + txt + "|"
                
        # 03 Dotted link
        elif shape == "dottedlink" :
            if txt == "" : 
                self.mermaid_txt += "-.->"
            else:
                self.mermaid_txt += "-." + txt + ".->"
        
        # 03 Thick link
        elif shape == "thicklink" :
            if txt == "" : 
                self.mermaid_txt += "==>"
            else:
                self.mermaid_txt += "==" + txt + "==>"
                
    # 03 Subgraphs        
    def subgraph(self, state = "", txt = ""):
        # Setting relation texts
        if state == "" :
            print("please set a shape and text for subgraph")
            
        if state == "start":
            self.mermaid_txt += "subgraph " + txt + ";"
            
        if state == "end" and txt == "":
            self.mermaid_txt += "end; "
            
    # 04 Setting Style 
    def add_node_color(self, node_id, color) :
        # Setting node Color
        self.mermaid_txt += "\n style " + node_id + " fill:" + color + ";"

In [7]:
mc = Mermaid_creator()
mc.subgraph(state = "start", txt = "123456789")

In [8]:
mc = Mermaid_creator()

mc.subgraph(state = "start", txt = "123456789")
mc.add_node(shape = "rectangular", txt = "First Node", node_id = "1")
mc.add_link(shape = "arrowhead", txt = "")
mc.add_node(shape = "rectangular", txt = "Second Node", node_id = "2")
mc.close_sentence()

mc.add_node(shape = "roundedges", txt = "Third Node", node_id = "3")
mc.add_link(shape = "arrowhead", txt = "Some relations")
mc.add_node(shape = "rectangular", txt = "Second Node", node_id = "2")
mc.close_sentence()
mc.subgraph(state = "end")

mc.add_node(shape = "rectangular", txt = "Second Node", node_id = "2")
mc.add_link(shape = "dottedlink", txt = "table connect")
mc.add_node(shape = "diamond", txt = "Forth Node", node_id = "4")
mc.close_sentence()

mc.add_node(node_id = "4")
mc.add_link(shape = "dottedlink", txt = "table connect")
mc.add_node(node_id = "5")
mc.close_sentence()

In [9]:
mc.mermaid_txt

'graph TD;subgraph 123456789;1[First Node]-->2[Second Node];3(Third Node)-->|Some relations|2[Second Node];end; 2[Second Node]-.table connect.->4{Forth Node};4-.table connect.->5;'

In [10]:
sas_query.from_to_property
sas_query.from_to_query
sas_query.node_property

[['node_id',
  'query_id',
  'subq_name',
  'state',
  'token_tag',
  'state_value',
  'state_alias',
  'state_parentname',
  'state_realname'],
 ['1',
  '1',
  'MAIN',
  'FROM',
  'value',
  'Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)',
  '',
  '',
  ''],
 ['2',
  '1',
  'MAIN',
  'FROM',
  'token',
  'alias:, Database:Meow, Table:POL_ALL_X_01',
  '',
  'Meow',
  'POL_ALL_X_01'],
 ['3',
  '1',
  'MAIN',
  'FROM',
  'token',
  'alias:t2, Database:QUERY, Table:DTAAB001_Q',
  't2',
  'QUERY',
  'DTAAB001_Q'],
 ['4',
  '1',
  'MAIN',
  'WHERE',
  'value',
  'WHERE t1.CREATE_DATE > t2.OCR_DATE ;',
  '',
  '',
  ''],
 ['5',
  '1',
  'MAIN',
  'SELECT',
  'value',
  'DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO',
  '',
  '',
  ''],
 ['6',
  '1',
  'MAIN',
  'TABLE',
  'token',
  'MEOW.PAST_CLAIM_APLY_NO',
  '',
  'MEOW',
  'PAST_CLAIM_APLY_NO'],
 ['7',
  '2',
  'MAIN',
  'FROM',
  'value',
  'MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_

In [43]:
import pandas as pd
node_property

,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
0,1,1,MAIN,FROM,value,Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q ...,,,
1,2,1,MAIN,FROM,token,"alias:, Database:Meow, Table:POL_ALL_X_01",,Meow,POL_ALL_X_01
2,3,1,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAB001_Q",t2,QUERY,DTAAB001_Q
3,4,1,MAIN,WHERE,value,WHERE t1.CREATE_DATE > t2.OCR_DATE ;,,,
4,5,1,MAIN,SELECT,value,"DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO",,,
5,6,1,MAIN,TABLE,token,MEOW.PAST_CLAIM_APLY_NO,,MEOW,PAST_CLAIM_APLY_NO
6,7,2,MAIN,FROM,value,MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DT...,,,
7,8,2,MAIN,FROM,token,"alias:t1, Database:MEOW, Table:PAST_CLAIM_APLY_NO",t1,MEOW,PAST_CLAIM_APLY_NO
8,9,2,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAA020_AE",t2,QUERY,DTAAA020_AE
9,10,2,MAIN,SELECT,value,"DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY...",,,


In [211]:
node_property = pd.DataFrame(sas_query.node_property[1:], columns = sas_query.node_property[0])
node_property[(node_property["node_id"] == "3") & (node_property["token_tag"] == "token")]

,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
2,3,1,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAB001_Q",t2,QUERY,DTAAB001_Q


In [90]:
node_property[node_property["node_id"] == "3"]["node_id"].tolist()

['3']

In [238]:
import pandas as pd
class Sqltoflowchart():
    
    def __init__(self, from_to_node, from_to_query, node_property):
        self.from_to_node = from_to_node
        self.from_to_query = from_to_query
        self.node_property = pd.DataFrame(node_property[1:], columns= node_property[0])
        self.mc = Mermaid_creator()
    
    # 00_1 regrex Setting (upper and lower no different)
    def upper_lower_regrex(self, string):
        regex_txt = ""
        for c in string:
            if c == " " : 
                regex_txt += " "
            else:
                regex_txt += "[" + c.lower() + c.upper() + "]"
        return(regex_txt)
    
    # 01 Create node txt
    def Create_nodetxt(self, node_id, node):
        token_tag = node["token_tag"].tolist()[0]
        node_value = ""
        node_value += "<b>" + node["state"].tolist()[0] + "</b>"
        
        if token_tag == "token" :
            node_value += "<br>" + node["state_parentname"].tolist()[0] + "." + node["state_realname"].tolist()[0] + ", " + node["state_alias"].tolist()[0]
        
        ## Add <center> </center>
        node_value = "<center>" + node_value + "</center>"
        
        return node_value
    
    # 02 Create relation txt
    def Create_relationtxt(self, node_id, node):
        token_tag = node["token_tag"].tolist()[0]
        relation_value = ""
        
        if token_tag == "token" : 
            print("Empty relation (Class Sqltoflowchart - def Create_relationtxt)")
        else :
            relation_value = node[ "state_value" ].tolist()[0]
            relation_value = re.sub("\\(|\\)", " ", relation_value)
            relation_value = re.sub(";", "", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("INNER JOIN "), "<b>INNER JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("LEFT JOIN "), "<b>LEFT JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("RIGHT JOIN "), "<b>RIGHT JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("OUTER JOIN "), "<b>OUTER JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("CROSS JOIN "), "<b>CROSS JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("DISTINCT "), "<b>DISTINCT </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("ON "), "ON <br>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex(","), ", <br>", relation_value)
#             print("Need txt relation (Class Sqltoflowchart - def Create_relationtxt)")
        ## Add <center> </center>
        relation_value = "<center>" + relation_value + "</center>"
        
        return(relation_value)    
    
    #03 Create mermaid plot
    def mermaid_plot(self):
        #### 03_2 Initial Subgraph start drawing
        subgraph_sign = 0 # Record the subgraph , start->set 1, end->set 0
        
        
        #01 Node relation create
        for from_to in self.from_to_node :
            from_id = ""
            to_id = ""
            from_node = ""
            to_node = ""
            node_txt = ""
            relation = ""
            
            if from_to[0] == "from" :
                print("columnnames" + str(from_to))
                continue
            
            
            # 03_1 Initial setting from and to node setting 
            from_id = from_to[0]
            to_id = from_to[1]
            from_node = self.node_property[self.node_property["node_id"] == from_id]
            to_node = self.node_property[self.node_property["node_id"] == to_id]
            
            #03_2 Subgraph start drawing(from_node) ===============================
            
            if from_node["state"].tolist()[0] == "FROM" and from_node["token_tag"].tolist()[0] == "token" and subgraph_sign == 0 :
                subgraph_sign = 1
                self.mc.subgraph(state = "start", txt = "<needtobereplacetable>")
                
                
            #03_3 From node create (from_node)===============================
            node_txt = self.Create_nodetxt(node_id = from_id, node = from_node)
            if from_node["token_tag"].tolist()[0] == "token" :
                self.mc.add_node(shape = "roundedges", txt = node_txt, node_id = from_id)
            else : 
                self.mc.add_node(shape = "diamond", txt = node_txt, node_id = from_id)
            
            
            #03_4 From relation create (from_node)===============================
            relation_txt = self.Create_relationtxt(node_id = from_id, node = from_node)
            if from_node["token_tag"].tolist()[0] == "token" :
                self.mc.add_link(shape = "arrowhead", txt = "")
            else : 
                self.mc.add_link(shape = "arrowhead", txt = relation_txt)
            
            
            
            #03_5 To node create (to_node)===============================
            node_txt = self.Create_nodetxt(node_id = to_id, node = to_node)
            if to_node["token_tag"].tolist()[0] == "token" :
                self.mc.add_node(shape = "roundedges", txt = node_txt, node_id = to_id)
            else : 
                self.mc.add_node(shape = "diamond", txt = node_txt, node_id = to_id)
            
            #03_6 End Line ===============================
            self.mc.close_sentence()
            
            
            #03_7 Subgraph end drawing (to_node) ===============================
            ## 01 "TABLE" state, "token" token_tag setting end line
            ## 02 Replace the  mermaid_txt <needtobereplacetable> replace to TABLE name 
            if to_node["state"].tolist()[0] == "TABLE" and to_node["token_tag"].tolist()[0] == "token" and subgraph_sign == 1 :
                subgraph_sign = 0
                self.mc.subgraph(state = "end")
                self.mc.mermaid_txt = re.sub("\\<needtobereplacetable\\>", to_node["state_value"].tolist()[0], self.mc.mermaid_txt)
                print("replace relation (Class Sqltoflowchart - def Create_relationtxt)")
            
        #02 query relation create
        for from_to_query in self.from_to_query :
            from_id = ""
            to_id = ""
            if from_to_query[0] == "from" :
                print("columnnames" + str(from_to_query))
                continue
            print("columnnames" + str(from_to_query))
            # 04_1 Initial setting from and to node setting 
            from_id = from_to_query[0]
            to_id = from_to_query[1]
            
            self.mc.add_node(node_id = from_id)
            self.mc.add_link(shape = "dottedlink", txt = "")
            self.mc.add_node(node_id = to_id)
            self.mc.close_sentence()
            
    #04 Draw mermaid plot
    def mermaid_drawnode(self, keyword, token_tag, color):
        draw_nodes = self.node_property[(self.node_property["state_parentname"] == keyword) & (self.node_property["token_tag"] == token_tag)]
        for node_id in draw_nodes["node_id"]:
            self.mc.add_node_color(node_id, color)

In [239]:
sfc = Sqltoflowchart(sas_query.from_to_property,
sas_query.from_to_query,
sas_query.node_property)

In [214]:
sas_query.from_to_query

[['from', 'to'], ['6', '8']]

In [240]:
sfc.mermaid_plot()
sfc.mermaid_drawnode(keyword = "QUERY", token_tag = "token", color = "#28FF28")

columnnames['from', 'to']
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
replace relation (Class Sqltoflowchart - def Create_relationtxt)
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
replace relation (Class Sqltoflowchart - def Create_relationtxt)
columnnames['from', 'to']
columnnames['6', '8']


In [241]:
sfc.mc.mermaid_txt

'graph TD;subgraph MEOW.PAST_CLAIM_APLY_NO;2(<center><b>FROM</b><br>Meow.POL_ALL_X_01, </center>)-->1{<center><b>FROM</b></center>};3(<center><b>FROM</b><br>QUERY.DTAAB001_Q, t2</center>)-->1{<center><b>FROM</b></center>};1{<center><b>FROM</b></center>}-->|<center>Meow.POL_ALL_X_01 <b>INNER JOIN </b>QUERY.DTAAB001_Q t2 ON <br> t1.INSD_ID_SAS = t2.OCR_ID_SAS </center>|4{<center><b>WHERE</b></center>};4{<center><b>WHERE</b></center>}-->|<center>WHERE t1.CREATE_DATE > t2.OCR_DATE </center>|5{<center><b>SELECT</b></center>};5{<center><b>SELECT</b></center>}-->|<center><b>DISTINCT </b>t1.CONTRACT_NO, <br>t1.INSD_ID_SAS, <br>t2.APLY_NO</center>|6(<center><b>TABLE</b><br>MEOW.PAST_CLAIM_APLY_NO, </center>);end; subgraph WORK.PAST_CLAIM_ICDS;8(<center><b>FROM</b><br>MEOW.PAST_CLAIM_APLY_NO, t1</center>)-->7{<center><b>FROM</b></center>};9(<center><b>FROM</b><br>QUERY.DTAAA020_AE, t2</center>)-->7{<center><b>FROM</b></center>};7{<center><b>FROM</b></center>}-->|<center>MEOW.PAST_CLAIM_APLY_NO t

In [79]:
sas_query.from_to_property

[['from', 'to'],
 ['2', '1'],
 ['3', '1'],
 ['1', '4'],
 ['4', '5'],
 ['5', '6'],
 ['8', '7'],
 ['9', '7'],
 ['7', '10'],
 ['10', '11']]

In [75]:
regex_txt = ""
for c in "INNER JOIN ":
    if c == " " : 
        regex_txt += " "
    else:
        regex_txt += "[" + c.lower() + c.upper() + "]"
print(regex_txt)

[iI][nN][nN][eE][rR] [jJ][oO][iI][nN] 


In [78]:
sfc.Create_relationtxt(node_id = "7", node = sfc.node_property[sfc.node_property["node_id"] == "7"])

'MEOW.PAST_CLAIM_APLY_NO t1 <b>INNER JOIN </b>QUERY.DTAAA020_AE t2 ON <br> t1.APLY_NO = t2.APLY_NO  ;'

## mermaid plot 
### supermarkdown

In [153]:
re.sub("\\<needtobereplacetable\\>", "xxxx", "<needtobereplacetable>;")

'xxxx;'

In [64]:
re.sub( regex_txt, "", " FROM Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS) ")

' FROM Meow.POL_ALL_X_01 QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS) '

In [93]:
from SuperMarkdown import SuperMarkdown

supermd = SuperMarkdown.SuperMarkdown()

In [10]:
import markdown

text = """

~~~mermaid
graph LR
A[Zip file]-->B[\"R (abc) functions\"]
A[Zip file]-->C[manual<br>for R<br>functions]
A[Zip file]-->D[vingettes]
D[vingettes]-->E[data]
D[vingettes]-->F[figures]
D[vingettes]-->G((R<br>markdown<br>file))
A[Zip file]-->H((README<br>file))

style B fill:red
~~~
"""

html = markdown.markdown(text, extensions=['md_mermaid'])

print(html)

<div class="mermaid">
graph LR
A[Zip file]-->B["R (abc) functions"]
A[Zip file]-->C[manual<br>for R<br>functions]
A[Zip file]-->D[vingettes]
D[vingettes]-->E[data]
D[vingettes]-->F[figures]
D[vingettes]-->G((R<br>markdown<br>file))
A[Zip file]-->H((README<br>file))

style B fill:red
</div>

<script>mermaid.initialize({startOnLoad:true});</script>
